<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/openai_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 使用查询引擎工具的OpenAI代理

这个项目旨在使用OpenAI的GPT-3代理来构建一个简单的查询引擎工具。我们将使用Python编程语言和OpenAI的GPT-3 API来实现这个项目。我们将使用OpenAI的GPT-3代理来回答用户提出的问题，并将其集成到一个简单的查询引擎工具中。


## 构建查询引擎工具


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-agent-openai

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

下载数据


In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
if not index_loaded:    # 加载数据    lyft_docs = SimpleDirectoryReader(        input_files=["./data/10k/lyft_2021.pdf"]    ).load_data()    uber_docs = SimpleDirectoryReader(        input_files=["./data/10k/uber_2021.pdf"]    ).load_data()    # 构建索引    lyft_index = VectorStoreIndex.from_documents(lyft_docs)    uber_index = VectorStoreIndex.from_documents(uber_docs)    # 持久化索引    lyft_index.storage_context.persist(persist_dir="./storage/lyft")    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

```python
import gym
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
```


In [ ]:
from llama_index.agent.openai import OpenAIAgent

In [ ]:
agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

## 让我们试一试！


In [ ]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

=== Calling Function ===
Calling function: lyft_10k with args: {
  "input": "What was Lyft's revenue growth in 2021?"
}
Got output: 
Lyft's revenue growth in 2021 was 36%.
=== Calling Function ===
Calling function: uber_10k with args: {
  "input": "What was Uber's revenue growth in 2021?"
}
Got output: 
Uber's revenue growth in 2021 was 57%.
Assistant: Lyft's revenue growth in 2021 was 36%, while Uber's revenue growth in 2021 was 57%.

